### Packages

In [15]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
!pip install arch -U
from arch import arch_model
import seaborn as sns
!pip install yfinance -U
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

Requirement already up-to-date: arch in /opt/conda/lib/python3.7/site-packages (4.14)
Requirement already up-to-date: yfinance in /opt/conda/lib/python3.7/site-packages (0.1.54)


### Loading the data

In [16]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2018-01-29", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

[*********************100%***********************]  4 of 4 completed


In [17]:
df_comp = raw_data.copy()

In [18]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [19]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

### Creating Returns

In [20]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

### Splitting the Data

In [21]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

### Fitting a Model

In [22]:
!pip install pmdarima

In [23]:
from pmdarima.arima import auto_arima

In [24]:
model_auto = auto_arima(df.ret_ftse[1:])

In [25]:
model_auto

ARIMA(maxiter=50, method='lbfgs', order=(4, 0, 5), out_of_sample_size=0,
      scoring='mse', scoring_args=None, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=False, trend=None,
      with_intercept=True)

In [26]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5019
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -7882.658
Date:                Fri, 01 May 2020   AIC                          15787.316
Time:                        12:28:21   BIC                          15859.047
Sample:                             0   HQIC                         15812.452
                               - 5019                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0309      0.025      1.246      0.213      -0.018       0.080
ar.L1          0.0135      0.082      0.165      0.869      -0.147       0.174
ar.L2         -0.6690      0.077     -8.645      0.000      -0.821      -0.517
ar.L3         -0.1616      0.072     -2.260      0.024      -0.302      -0.021
ar.L4          0.1898      0.074      2.553      0.011       0.044       0.335
ma.L1         -0.0384      0.081     -0.471      0.637      -0.198       0.121
ma.L2          0.6205      0.078      7.933      0.000       0.467       0.774
ma.L3          0.0592      0.069      0.858      0.391      -0.076       0.194
ma.L4         -0.1836      0.073     -2.510      0.012      -0.327      -0.040
ma.L5         -0.1063      0.010    -11.130      0.000      -0.125      -0.088
sigma2         1.3541      0.015     91.128      0.000       1.325       1.383
===================================================================================
Ljung-Box (Q):                       67.77   Jarque-Bera (JB):              6360.06
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               2.00   Skew:                            -0.19
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.50
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## Important Arguments

In [27]:
model_auto = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample = int(len(df_comp)*0.2))


# !!! Important Note: In pdmarima v1.5.2, out_of_sample_size is replaced with out_of_sample, so make sure to use the latter!


# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [28]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                 6275
Model:             SARIMAX(2, 0, 2)x(1, 0, [1], 5)   Log Likelihood               -6340.064
Date:                             Fri, 01 May 2020   AIC                          12704.127
Time:                                     12:31:41   BIC                          12785.059
Sample:                                 01-11-1994   HQIC                         12732.169
                                      - 01-29-2018                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0047      0.009     -0.550      0.582      -0.022       0.012
drift      -1.345e-06   2.71e-06     -0.497      0.619   -6.65e-06    3.96e-06
ret_spx        0.0959      0.006     16.961      0.000       0.085       0.107
ret_dax        0.5581      0.005    113.715      0.000       0.549       0.568
ret_nikkei     0.0703      0.004     16.560      0.000       0.062       0.079
ar.L1         -0.1933      0.100     -1.927      0.054      -0.390       0.003
ar.L2          0.5083      0.059      8.681      0.000       0.394       0.623
ma.L1          0.0861      0.101      0.856      0.392      -0.111       0.283
ma.L2         -0.5371      0.056     -9.575      0.000      -0.647      -0.427
ar.S.L5        0.0698      0.339      0.206      0.837      -0.595       0.734
ma.S.L5       -0.0981      0.339     -0.289      0.772      -0.763       0.567
sigma2         0.4404      0.004     99.154      0.000       0.432       0.449
===================================================================================
Ljung-Box (Q):                       84.55   Jarque-Bera (JB):             15298.73
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.54   Skew:                             0.25
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.63
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## AUTO ARIMA With Arguments

In [29]:
model_arima = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5, max_order = None,
                        max_p = 7, max_q = 7, max_d = 2,
                        max_P = 4, max_Q = 4, max_D = 2,
                        maxiter = 50, alpha = 0.05, n_jobs = -1,
                        trend = 'ct', information_criterion = "oob",
                        out_of_sample_size = int(len(df_comp)*0.2))

In [31]:
model_arima.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                        
==============================================================================================
Dep. Variable:                                      y   No. Observations:                 6275
Model:             SARIMAX(3, 0, 1)x(1, 0, [1, 2], 5)   Log Likelihood               -6347.774
Date:                                Fri, 01 May 2020   AIC                          12721.548
Time:                                        13:22:29   BIC                          12809.225
Sample:                                             0   HQIC                         12751.927
                                               - 6275                                         
Covariance Type:                                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0067      0.017     -0.401      0.689      -0.039       0.026
drift      -7.364e-07   5.08e-06     -0.145      0.885   -1.07e-05    9.21e-06
x1             0.0854      0.006     13.757      0.000       0.073       0.098
x2             0.5634      0.005    103.429      0.000       0.553       0.574
x3             0.0739      0.005     15.675      0.000       0.065       0.083
ar.L1         -0.0583      0.127     -0.457      0.647      -0.308       0.191
ar.L2         -0.0166      0.018     -0.937      0.349      -0.051       0.018
ar.L3         -0.0686      0.010     -6.706      0.000      -0.089      -0.049
ma.L1         -0.0613      0.128     -0.479      0.632      -0.312       0.189
ar.S.L5       -0.0113      0.183     -0.062      0.951      -0.370       0.348
ma.S.L5       -0.0216      0.183     -0.119      0.906      -0.379       0.336
ma.S.L10      -0.0550      0.012     -4.540      0.000      -0.079      -0.031
sigma2         0.4870      0.005     89.705      0.000       0.476       0.498
===================================================================================
Ljung-Box (Q):                       64.75   Jarque-Bera (JB):             12878.04
Prob(Q):                              0.01   Prob(JB):                         0.00
Heteroskedasticity (H):               0.55   Skew:                             0.22
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.00
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## The 'drift' value is the value for the Linear component of the trend